# Clustering Crypto

In [7]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [9]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('./resources/crypto_data.csv', index_col=0)
crypto_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './resources/crypto_data.csv'

In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_trade_df = crypto_df.query("IsTrading == True")
print(crypto_trade_df.shape)
crypto_trade_df.head(10)

In [ ]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_trade_df = crypto_df.query("IsTrading == True")
print(crypto_trade_df.shape)
crypto_trade_df.head(10)

In [ ]:
# Remove the "IsTrading" column. 
crypto_trade_df = crypto_trade_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']]
print(crypto_trade_df.shape)
crypto_trade_df.head(10)

In [ ]:
# Remove rows that have at least 1 null value.
crypto_trade_df = crypto_trade_df.dropna()
print(crypto_trade_df.shape)
crypto_trade_df.head(10)

In [ ]:
# Keep the rows where coins are mined.
crypto_df = crypto_trade_df.query('TotalCoinsMined >= 1')
print(crypto_df.shape)
crypto_df.head(10)

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = crypto_df[['CoinName']]
names_df.shape
names_df.head()

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_filter_df = crypto_df[['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']]
print(crypto_filter_df.shape)
crypto_filter_df.head(10)

In [ ]:
# Use get_dummies() to create variables for text features.
crypto_filter_dum = pd.get_dummies(crypto_filter_df, columns=["Algorithm", "ProofType"])
print(crypto_filter_dum.shape)
crypto_filter_dum.head(10)

In [ ]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler()
loans_data_scaled = data_scaler.fit_transform(crypto_filter_dum)
loans_data_scaled

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(loans_data_scaled)
crypto_pca

In [ ]:
# Create a DataFrame with the three principal components.
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=names_df.index
)
print(df_crypto_pca.shape)
# YOUR CODE HERE
df_crypto_pca.head(10)

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)
predictions

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_filter_df.merge(df_crypto_pca, left_index=True, right_index=True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(names_df)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x = "PC 1",
    y = "PC 2",
    z = "PC 3",
    color = "class",
    symbol = "class",
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
    width = 800
)
fig.update_layout(legend = dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], sortable=True, selectable=True)

In [ ]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df)} tradable cryptocurrencies.')

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_data = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
scaled_data

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    data=scaled_data, columns=["TotalCoinSupply", "TotalCoinsMined"], index=names_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = names_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["class"] = model.labels_

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x = "TotalCoinsMined",
    y = "TotalCoinSupply",
    hover_cols = ["CoinName"],
    by = "class"
)